In [1]:
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing,utils

In [2]:
import json

In [3]:
with open('intents.json','r') as intents:
    intents= json.load(intents)

In [4]:
questions = []
responses = []

In [5]:
# Iterate over the intents
for intent in intents['intents']:
    # Iterate over the patterns and responses in each intent
    for pattern, response in zip(intent['patterns'], intent['responses']):
        questions.append(pattern)  # Put the intent in the dataset before pattern
        responses.append(response)  # Put the intent in the dataset before response

In [6]:
print(questions)

['Hi', 'How are you?', 'Is anyone there?', 'cya', 'see you', 'bye bye', 'See you later', 'what is the name of your developers', 'name', 'your name', 'do you have a name', 'what are you called', 'timing of college', 'more info', 'list of courses', 'information about fee', 'where is the college located', 'hostel facility', 'events organised', 'document to bring', 'size of campus', 'Syllabus for IT', 'is there any library', 'how is college infrastructure', 'food facilities', 'food menu', 'What is college placement', 'Who is HOD', 'Who is computer HOD', 'Who is extc HOD', 'what is the name of principal', 'exam dates', 'what is the process of admission', 'scholarship', 'What facilities college provide', 'max number of students', 'college dress code', 'what are the different committe in college', 'I love you', 'fuck', 'bitch', 'holidays', 'sports and games', 'okk', 'okie', 'what can you do', 'what are the thing you can do', 'ragging', 'hod']


In [7]:
print(responses)

['Hello!', 'Good to see you again!', 'Hi there, how can I help?', 'Sad to see you go :(', 'Talk to you later', 'Goodbye!', 'Come back soon', 'College students', 'You can call me Mind Reader.', "I'm Mind Reader", 'I am a Chatbot.', 'I am your helper', 'College is open 8am-5pm Monday-Saturday!', 'You can contact at: NUMBER', 'Our university offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering.', 'For Fee detail visit <a target="_blank" href="LINK"> here</a>', '<a target="_blank" href="ADD YOU GOOGLE MAP LINK HERE"> here</a>', 'For hostel detail visit <a target="_blank" href="ADD YOUR HOSTEL DETAIL PDF LINK OR ANY INFORMATION LINK OR ADD YOU OWN ANSWERS"> here</a>', 'For event detail visit <a target="_blank" href="ADD YOUR FUNCTIONS LINK OR YOUR OWN RESPONSE"> here</a>', 'To know more about document required visit <a target="_blank" href="ADD LINK OF ADMISSION GUIDANCE DOCUMENT FROM YOUR UNIVERSITY WEBSITE

In [8]:
answers_with_tags = []
for i in range(len(responses)):
    if type(responses[i]) == str:
        answers_with_tags.append(responses[i])
    else:
        responses.pop(i)

answers = []
for i in range(len(answers_with_tags)) :
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index)+1

In [9]:
VOCAB_SIZE

274

In [10]:
vocabulary=tokenizer.word_index
print(vocabulary)

{'start': 1, 'end': 2, 'a': 3, 'you': 4, 'is': 5, 'college': 6, 'your': 7, 'to': 8, 'of': 9, 'for': 10, 'and': 11, 'target': 12, 'blank': 13, 'href': 14, 'link': 15, 'here': 16, 'the': 17, 'i': 18, 'what': 19, 'hod': 20, 'can': 21, 'visit': 22, 'are': 23, 'more': 24, 'university': 25, 'engineering': 26, 'name': 27, 'our': 28, 'add': 29, 'have': 30, 'who': 31, 'detail': 32, 'or': 33, 'with': 34, 'be': 35, 'there': 36, 'see': 37, 'information': 38, 'about': 39, 'document': 40, 'it': 41, 'library': 42, 'number': 43, 'students': 44, 'different': 45, 'am': 46, 'website': 47, 'how': 48, 'do': 49, 'hostel': 50, 'campus': 51, 'any': 52, 'food': 53, 'facilities': 54, 'placement': 55, 'computer': 56, 'extc': 57, 'admission': 58, 'in': 59, 'ragging': 60, 'good': 61, 'own': 62, 'know': 63, 'has': 64, 'one': 65, 'available': 66, 'we': 67, 'on': 68, 'if': 69, 'all': 70, 'xyz': 71, 'department': 72, 'branch': 73, 'appropriate': 74, 'academic': 75, 'details': 76, 'provides': 77, 'please': 78, 'hi': 79

In [11]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

In [12]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)
print(tokenized_questions)

(49, 7) 7
[[79], [48, 23, 4], [5, 117, 36], [118], [37, 4], [80, 80], [37, 4, 81], [19, 5, 17, 27, 9, 7, 119], [27], [7, 27], [49, 4, 30, 3, 27], [19, 23, 4, 120], [121, 9, 6], [24, 122], [123, 9, 124], [38, 39, 82], [125, 5, 17, 6, 126], [50, 127], [128, 129], [40, 8, 130], [131, 9, 51], [83, 10, 41], [5, 36, 52, 42], [48, 5, 6, 84], [53, 54], [53, 85], [19, 5, 6, 55], [31, 5, 20], [31, 5, 56, 20], [31, 5, 57, 20], [19, 5, 17, 27, 9, 86], [132, 133], [19, 5, 17, 134, 9, 58], [87], [19, 54, 6, 88], [135, 43, 9, 44], [6, 136, 137], [19, 23, 17, 45, 89, 59, 6], [18, 138, 4], [139], [140], [141], [90, 11, 142], [143], [144], [19, 21, 4, 49], [19, 23, 17, 145, 4, 21, 49], [60], [20]]


In [13]:
print(encoder_input_data.shape)

(49, 7)


In [14]:
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_input_data = np.array(padded_answers)

In [15]:
print(decoder_input_data.shape)

(49, 31)


In [16]:
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)

In [17]:
print(decoder_output_data.shape)

(49, 31, 274)


In [18]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 7)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 31)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 7, 200)            │          54,800 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal (NotEqual)          │ (None, 7)                 │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, 31, 200)           │          54,800 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 200), (None,      │         320,800 │ embedding[0][0],           │
│                               │ 200), (None, 200)]        │                 │ not_equal[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, 31, 200), (None,  │         320,800 │ embedding_1[0][0],         │
│                               │ 200), (None, 200)]        │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 31, 274)           │          55,074 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 806,274 (3.08 MB)

 Trainable params: 806,274 (3.08 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 
model.save( 'model.keras' )

Epoch 1/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - loss: 5.6131
Epoch 2/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - loss: 5.6055
Epoch 3/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - loss: 5.5993
Epoch 4/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - loss: 5.5934
Epoch 5/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - loss: 5.5873
Epoch 6/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - loss: 5.5806
Epoch 7/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - loss: 5.5728
Epoch 8/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - loss: 5.5633
Epoch 9/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - loss: 5.5506
Epoch 10/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - loss: 5.5311
Epoch 11/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step - loss: 5.4939
Epoch 12/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - loss: 5.4025
Epoch 13/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - loss: 5.2228
Epoch 14/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - loss: 5.0776
Epoch 15/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - loss: 5.0071
Epoch

In [20]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [21]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [22]:
print(tokenizer.word_index)

{'start': 1, 'end': 2, 'a': 3, 'you': 4, 'is': 5, 'college': 6, 'your': 7, 'to': 8, 'of': 9, 'for': 10, 'and': 11, 'target': 12, 'blank': 13, 'href': 14, 'link': 15, 'here': 16, 'the': 17, 'i': 18, 'what': 19, 'hod': 20, 'can': 21, 'visit': 22, 'are': 23, 'more': 24, 'university': 25, 'engineering': 26, 'name': 27, 'our': 28, 'add': 29, 'have': 30, 'who': 31, 'detail': 32, 'or': 33, 'with': 34, 'be': 35, 'there': 36, 'see': 37, 'information': 38, 'about': 39, 'document': 40, 'it': 41, 'library': 42, 'number': 43, 'students': 44, 'different': 45, 'am': 46, 'website': 47, 'how': 48, 'do': 49, 'hostel': 50, 'campus': 51, 'any': 52, 'food': 53, 'facilities': 54, 'placement': 55, 'computer': 56, 'extc': 57, 'admission': 58, 'in': 59, 'ragging': 60, 'good': 61, 'own': 62, 'know': 63, 'has': 64, 'one': 65, 'available': 66, 'we': 67, 'on': 68, 'if': 69, 'all': 70, 'xyz': 71, 'department': 72, 'branch': 73, 'appropriate': 74, 'academic': 75, 'details': 76, 'provides': 77, 'please': 78, 'hi': 79

In [23]:
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

def load_intents_from_json(file_path):
    with open(file_path, 'r') as file:
        intents = json.load(file)
    return intents

def extract_keywords(question):
    # Tokenize the question into words
    words = word_tokenize(question.lower())
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    keywords = [word for word in words if word not in stop_words]
    return keywords

def identify_intent(question, intents):
    keywords = extract_keywords(question)
    keyword_counts = Counter(keywords)
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            pattern_keywords = extract_keywords(pattern)
            if any(keyword_counts[key] > 0 for key in pattern_keywords):
                return intent['tag']
    return None  # Return None if no intent is found

# Load intents from JSON file
intents = load_intents_from_json('intents.json')

# Example usage
question = "Hi, how are you?"
intent_tag = identify_intent(question, intents)
if intent_tag:
    print(f"The intent for the question '{question}' is '{intent_tag}'.")
else:
    print("No matching intent found for the question.")


The intent for the question 'Hi, how are you?' is 'greeting'.


In [32]:
enc_model , dec_model = make_inference_models()

for _ in range(4):
    ask=input('Enter question: ' )
    intent=identify_intent(ask,intents)
    states_values = enc_model.predict( str_to_tokens( ask ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict( [empty_target_seq]+states_values ,verbose =0)
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoder_translation[_] )

Enter question:  hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
Hello hello good end


Enter question:  hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Hello to see you help end


Enter question:  bye


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
see you later end


Enter question:  your name is


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
 Mind reader end
